In [5]:
using JuMP
#using GLPKMathProgInterface
using CPLEX

# Modelo simple

\begin{align}
\text{minimize} \qquad & x+y \\
 \text{subject to} \quad \quad & x+y \geq 1 \\
 \qquad \qquad & x \geq 0, y \geq 0 \\
 \qquad \qquad & x,y \in \mathbb{R}
\end{align}



In [26]:
#m = Model(solver = GLPKSolverMIP())
#m = Model(solver = GLPKSolverLP())

#creamos un modelo
m = Model(solver = CplexSolver())

#y sus variables
@variable(m, x >= 0 )
@variable(m, y >= 0 )

#función objetivo a minimizar
@objective(m, Min, x + y )

#restricción
@constraint(m, x + y >= 1.0 )

#escribimos el modelo
print(m)

#resolvemos
status = solve(m)

#mostramos resultados
println("Status: ",status," Objective value: ", getobjectivevalue(m))
println("x = ", getvalue(x))
println("y = ", getvalue(y))

Min x + y
Subject to
 x + y ≥ 1
 x ≥ 0
 y ≥ 0
Tried aggregator 1 time.
LP Presolve eliminated 1 rows and 2 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)
Status: Optimal Objective value: 1.0
x = 0.0
y = 1.0
CPLEX Error  3003: Not a mixed-integer problem.


# Un programa lineal en formato estándar

Un PL en formato estándar se expresa como:

$$
\begin{align}
& \text{min} && c^T x \\
& \text{subject to} && A x = b \\
&                   && x \succeq 0 \\
&                   && x \in \mathbb{R}^n
\end{align}
$$


In [26]:
#Datos
#----------

#INPUT DATA
#----------

c=[2500; 4000; 0; 0; 0] 
b=[200; 100; 750]
A=[1 0 1 0 0; 0 1 0 1 0; 3 5 0 0 1]
m, n = size(A) # m = filas n = columnas

(3,5)

In [28]:
modelo = Model(solver = CplexSolver())
@variable(modelo, x[1:n] >= 0) # Models x >=0
for i in 1:m # filas
    @constraint(modelo, sum(A[i, j]*x[j] for j in 1:n) == b[i]) # i-ésima restricción 
end 
@objective(modelo,Min,sum(c[i]*x[i] for i in 1:n))
status = solve(modelo) # solves the model  

Tried aggregator 1 time.
LP Presolve eliminated 3 rows and 5 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.00 ticks)
CPLEX Error  3003: Not a mixed-integer problem.


:Optimal

# Un modelo algo más complejo

Obviamente, el modelo anterior no tiene mayor dificultad. Alternativamente podríamos intentar resolver algún modelo con algo más de historia (como por ejemplo el Uncapacitated Lot Sizing Problem).

\begin{align}
\text{minimize} \qquad & \sum_{t\in T} c_s s_t+ \sum_{t\in T} c_K y_t \\
 \text{subject to} \quad \quad & s_{t-1} + x_t = s_t + d_t \\
 \qquad \qquad & x_t \leq M y_t \\
 \qquad \qquad & s_0 = 0 \\
 \qquad \qquad & x_t, s_t \in \mathbb{R}^+ \forall t\in T\\
 \qquad \qquad & y_t \in \{0,1\}
\end{align}

$c_s$ y $c_K$ son los costes de inventario (unidad por periodo) y de realizar un pedido en un periodo y $d_t$ la demanda de cada periodo. El conjunto de periodos es $T$. $x_t$ es la cantidad que se recibe en el periodo $t$, mientras que $y_t$ equivale a si se realiza un pedido en el periodo $t$ o no y $s_t$ es el inventario al final del periodo $t$.

El objetivo minimiza la suma de costos de inventario y pedido (el costo por unidad no se tiene en cuenta porque la solución óptima siempre compra exactamente la demanda, $\sum_{t\in T} d_t$.

La primera restricción asegura el cumplimiento de la demanda de cada periodo. La segunda restricción asegura que sólo se compran unidades si se ha realizado un pedido ($M$ es un valor lo suficientemente grande, en este caso basta con $\sum_{t\in T} d_t$). La tercera restricción inicializa el inventario y el resto de restricciones definen el problema.

In [13]:
T=10
d=[5; 10; 7; 11; 13; 14; 2; 8; 17; 21]
cK=50
cs=2
M=sum(d)

108

In [24]:
modelo=Model(solver = CplexSolver())
@variable(modelo, x[1:T]>= 0)
@variable(modelo, y[1:T],Bin)
@variable(modelo, s[0:T]>=0)

#función objetivo a minimizar
@objective(modelo, Min, sum(cs*s[t] for t in 1:T)+sum(cK*y[t] for t in 1:T))

for t in 1:T
    @constraint(modelo,s[t-1]+x[t]==d[t]+s[t])
end
for t in 1:T
    @constraint(modelo,x[t]<=M*y[t])
end
@constraint(modelo,s[0]==0)

#escribimos el modelo
print(modelo)

#resolvemos
status = solve(modelo)

#mostramos resultados
println("**** Status: ",status," Objective value: ", getobjectivevalue(modelo))
println("**** x = ", getvalue(x))
println("**** y = ", getvalue(y))
println("**** s = ", getvalue(s))

Min 2 s[1] + 2 s[2] + 2 s[3] + 2 s[4] + 2 s[5] + 2 s[6] + 2 s[7] + 2 s[8] + 2 s[9] + 2 s[10] + 50 y[1] + 50 y[2] + 50 y[3] + 50 y[4] + 50 y[5] + 50 y[6] + 50 y[7] + 50 y[8] + 50 y[9] + 50 y[10]
Subject to
 s[0] + x[1] - s[1] = 5
 s[1] + x[2] - s[2] = 10
 s[2] + x[3] - s[3] = 7
 s[3] + x[4] - s[4] = 11
 s[4] + x[5] - s[5] = 13
 s[5] + x[6] - s[6] = 14
 s[6] + x[7] - s[7] = 2
 s[7] + x[8] - s[8] = 8
 s[8] + x[9] - s[9] = 17
 s[9] + x[10] - s[10] = 21
 x[1] - 108 y[1] ≤ 0
 x[2] - 108 y[2] ≤ 0
 x[3] - 108 y[3] ≤ 0
 x[4] - 108 y[4] ≤ 0
 x[5] - 108 y[5] ≤ 0
 x[6] - 108 y[6] ≤ 0
 x[7] - 108 y[7] ≤ 0
 x[8] - 108 y[8] ≤ 0
 x[9] - 108 y[9] ≤ 0
 x[10] - 108 y[10] ≤ 0
 s[0] = 0
 x[i] ≥ 0 ∀ i ∈ {1,2,…,9,10}
 y[i] ∈ {0,1} ∀ i ∈ {1,2,…,9,10}
 s[i] ≥ 0 ∀ i ∈ {0,1,…,9,10}
Found incumbent of value 378.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 3 rows and 3 columns.
Reduced MIP has 18 rows, 28 columns, and 45 nonzeros.
Reduced MIP has 9 binaries, 0 generals, 0 SO